In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
import joblib



In [ ]:
df = pd.read_csv("hospital_deterioration_ml_ready.csv")

print(df.shape)
print(df['deterioration_next_12h'].value_counts())


(417866, 22)
deterioration_next_12h
0    395277
1     22589
Name: count, dtype: int64


In [ ]:
X = df.drop(columns=["deterioration_next_12h"])
y = df["deterioration_next_12h"]

# Encode categorical features
X = pd.get_dummies(X, drop_first=True)



In [ ]:
df = df.dropna(subset=["deterioration_next_12h"])



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = RandomForestClassifier(
    n_estimators=300,
    max_depth=20,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_prob))


              precision    recall  f1-score   support

           0       0.98      0.99      0.98     79056
           1       0.70      0.60      0.65      4518

    accuracy                           0.96     83574
   macro avg       0.84      0.79      0.81     83574
weighted avg       0.96      0.96      0.96     83574

AUC: 0.9703301287530174


In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=(y_train.value_counts()[0] / y_train.value_counts()[1]),
    objective="binary:logistic",
    eval_metric="auc",
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_train, y_train)

xgb_pred = xgb_model.predict(X_test)
xgb_prob = xgb_model.predict_proba(X_test)[:, 1]

print("\n=== XGBoost Results ===")
print(classification_report(y_test, xgb_pred))
print("ROC-AUC:", roc_auc_score(y_test, xgb_prob))


=== XGBoost Results ===
              precision    recall  f1-score   support

           0       0.99      0.95      0.97     79056
           1       0.50      0.89      0.64      4518

    accuracy                           0.95     83574
   macro avg       0.75      0.92      0.80     83574
weighted avg       0.97      0.95      0.95     83574

ROC-AUC: 0.9785415585404004


In [ ]:
print("\nPredicted risks from Random Forest:")
for i, r in enumerate(y_prob[:10]):
    print(f"Patient {i+1}: risk = {r:.3f}")

print("\nPredicted risks from XGBoost:")
for i, r in enumerate(xgb_prob[:10]):
    print(f"Patient {i+1}: risk = {r:.3f}")



Predicted risks from Random Forest:
Patient 1: risk = 0.347
Patient 2: risk = 0.405
Patient 3: risk = 0.025
Patient 4: risk = 0.051
Patient 5: risk = 0.156
Patient 6: risk = 0.021
Patient 7: risk = 0.002
Patient 8: risk = 0.186
Patient 9: risk = 0.202
Patient 10: risk = 0.402

Predicted risks from XGBoost:
Patient 1: risk = 0.313
Patient 2: risk = 0.752
Patient 3: risk = 0.008
Patient 4: risk = 0.041
Patient 5: risk = 0.673
Patient 6: risk = 0.000
Patient 7: risk = 0.005
Patient 8: risk = 0.271
Patient 9: risk = 0.146
Patient 10: risk = 0.378
